[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/04-langchain-chat.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/04-langchain-chat.ipynb)

In [8]:
!pip install -qU \
  langchain==0.3.25 \
  langchain-community==0.3.25 \
  langchain-openai==0.3.22 \
  jinja2==3.1.6

We'll start by initializing the `ChatOpenAI` object. For this we'll need an [OpenAI API key](https://platform.openai.com/account/api-keys). Note that there is naturally a small cost to running this notebook due to the paid nature of OpenAI's API access.

In [9]:
from IPython.display import display, Markdown
import os
from getpass import getpass

# must enter API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "langchain-pinecone-io-walkthrough-chat"

Initialize the `ChatOpenAI` object. We'll set `temperature=0` to minimize randomness and make outputs repeatable.

In [10]:
from langchain_openai import ChatOpenAI

# Initialize the chat model
chat = ChatOpenAI(
    temperature=1.0,
    model='gpt-5-mini'
)

Chats with the Chat-GPT model `gpt-4.1-mini` are typically structured like so:

```
System: You are a helpful assistant.

User: Hi AI, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand string theory.
```

The final `"Assistant:"` without a response is what would prompt the model to continue the conversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

```json
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"},
    {"role": "user", "content": "I'd like to understand string theory."}
]
```

In LangChain there is a slightly different format. We use three *message* objects like so:

In [11]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create the message templates
system_template = "You are a helpful assistant."
human_template = "{input}"

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

# Create the chain
chain = prompt | chat | StrOutputParser()

# Test the chain
result = chain.invoke({"input": "Hi AI, how are you today?"})
print(result)

Hi — I'm doing well, thanks! How can I help you today?


The format is very similar, we're just swapping the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [12]:
# Create initial messages
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Hi AI, how are you today?"),
    ("ai", "I'm great thank you. How can I help you?"),
    ("human", "I'd like to understand string theory.")
])

# Create the chain using LCEL pipe syntax
chain = prompt | chat | StrOutputParser()

# Get response using LCEL
res = chain.invoke({})
print(res)

Great — I can explain at several levels. Before I start: what’s your background (high‑school physics, undergraduate physics, familiarity with quantum mechanics and general relativity, or mathematical physics)? That will let me tailor the level. Meanwhile, here’s a compact, clear overview plus a suggested learning path.

Short, nontechnical summary
- String theory replaces point particles with tiny one‑dimensional strings. Different vibrational modes of a string look like different particles (electrons, photons, gravitons).
- Because strings have extent, some mathematical problems that make combining quantum mechanics and general relativity inconsistent for point particles are softened, and a spin‑2 vibration mode naturally behaves like a graviton — so string theory naturally includes gravity.
- To be mathematically consistent, string theory needs extra spatial dimensions (usually 9 space + 1 time in superstring theory). Those extra dimensions are “compactified” (curled up) on very tiny

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [13]:
# For the follow-up question, we can extend the existing prompt
prompt.extend([
    ("ai", res),  # Previous AI response
    ("human", "Why do physicists believe it can produce a 'unified theory'?")
])

# Create the chain using LCEL pipe syntax
chain = prompt | chat | StrOutputParser()

# Get response
result = chain.invoke({})
result

'Short answer\nPhysicists consider string theory a promising candidate for a unified theory because it naturally contains both gravity and the other forces in the same framework, it replaces the problematic point‑particle picture that makes quantum gravity nonrenormalizable, and it passes several nontrivial consistency checks that no arbitrary theory would.\n\nWhy that is — the core reasons\n\n- Gravity comes out automatically.\n  - The quantum excitations of a closed string include a massless spin‑2 state. In low‑energy effective field theory this state has the same couplings as the graviton of general relativity, so Einstein’s equations arise as the low‑energy dynamics of string theory.\n\n- Point‑particle short‑distance problems are softened.\n  - A string has finite size, so interactions are “smeared out” over the string scale. That changes high‑energy behavior of scattering amplitudes and removes many of the ultraviolet divergences that plague point‑particle quantum gravity. Pertu

## New Prompt Templates

Alongside what we've seen so far there are also three new prompt templates that we can use. Those are the `SystemMessagePromptTemplate`, `AIMessagePromptTemplate`, and `HumanMessagePromptTemplate`.

These are simply an extension of [Langchain's prompt templates](https://www.pinecone.io/learn/series/langchain/langchain-prompt-templates/) that modify the returning "prompt" to be a `SystemMessage`, `AIMessage`, or `HumanMessage` object respectively.

For now, there are not a huge number of use-cases for these objects. However, they can be useful if:
- You want different types of response; AND
- The types of response should depend on a set of pre-determined input values; AND
- You want to save tokens by not explicitly specifying every possible type of input value in the prompts.

This will make more sense with an example. Suppose you want to tailor responses to people from a wide variety of countries. E.g. an LLM powered worldwide translator!

Some of the languages listed have been commented out as this is just an illustrative example, but the idea is that we can have many languages and dynamically alter the `HumanMessage` prompt so that we don't have to list all of them every time.

In [14]:
languages = [
    "English",
    "Esperanto",
    "Spanish",
    # "French",
    # "German",
    # "Italian",
    # "Portuguese",
    # "Dutch",
    # "Russian",
    # "Chinese (Simplified)",
    # "Chinese (Traditional)",
    # "Japanese",
    # "Korean",
    # "Arabic",
    # "Hindi",
    # "Turkish",
    # "Swedish",
    # "Danish",
    # "Norwegian",
    # "Finnish",
    # "Polish",
    # "Czech",
    # "Hungarian",
    # "Greek",
    # "Hebrew",
    # "Vietnamese",
    # "Thai"
]

First let's see what the prompt looks like with single example.

In [15]:
from langchain.prompts.chat import HumanMessagePromptTemplate

# Create the prompt template
human_template = HumanMessagePromptTemplate.from_template(
    "Translate this input <INPUT_START> {input} <INPUT_END>  into {language}. Do not include any other text in your response."
)
chat_prompt = ChatPromptTemplate.from_messages([human_template])

# Format with dynamic input
chat_prompt_value = chat_prompt.format_prompt(
    input="I hope when you come the weather will be clement.", # Extra points if you get the reference.
    language="Esperanto"
)

chat_prompt_value


ChatPromptValue(messages=[HumanMessage(content='Translate this input <INPUT_START> I hope when you come the weather will be clement. <INPUT_END>  into Esperanto. Do not include any other text in your response.', additional_kwargs={}, response_metadata={})])

Note that to use `HumanMessagePromptTemplate` as typical a prompt templates with the `.format_prompt` method, we needed to pass it through a `ChatPromptTemplate` object. This is case for all of the new chat-based prompt templates.

Using this we return a `ChatPromptValue` object. This can be formatted into a list or string like so:

In [16]:
chat_prompt_value.to_messages()

[HumanMessage(content='Translate this input <INPUT_START> I hope when you come the weather will be clement. <INPUT_END>  into Esperanto. Do not include any other text in your response.', additional_kwargs={}, response_metadata={})]

In [17]:
chat_prompt_value.to_string()

'Human: Translate this input <INPUT_START> I hope when you come the weather will be clement. <INPUT_END>  into Esperanto. Do not include any other text in your response.'

Okay, let's see this new approach in action with our list of languages.

In [18]:
from langchain.prompts.chat import SystemMessagePromptTemplate

# Create the prompt template
human_template = HumanMessagePromptTemplate.from_template(
    "Translate this input '{input}' into {language}. Do not include any other text in your response."
)
system_template = SystemMessagePromptTemplate.from_template("You are a helpful assistant.")

# Create the chain using LCEL pipe syntax
chain = (
    ChatPromptTemplate.from_messages([system_template, human_template])
    | chat
    | StrOutputParser()
)

# Loop through each language
for language in languages:
    print(f"\n=== Response in {language} ===")

    # Invoke the chain with our inputs
    result = chain.invoke({
        "input": "I hope when you come the weather will be clement.",
        "language": language
    })

    print(result)
    print("=" * 50)  # Separator for readability


=== Response in English ===
I hope that when you come, the weather will be clement.

=== Response in Esperanto ===
Mi esperas, ke kiam vi venos, la vetero estos milda.

=== Response in Spanish ===
Espero que cuando vengas haga buen tiempo.


Excellent!

As you can see, it's successfully translated into different languages based on our inputs, *and we didn't have to use unnecessary tokens by inserting the entire language list into the prompt.*

What if the outputs we need are more complicated? For example, what if the input information is technical information that needs to be formatted in a very specific way for the output?

E.g. Say that we want to:
1. Input technical information.
2. Only translate part of the technical information, not all of the text.
3. Maintain the same input structure in the output structure.

We can use the prompt templates approach for building an initial system message with a few examples for the chatbot to follow — few-shot training via examples. Let's see what that looks like.

In [19]:
# Create few-shot examples for technical content formatting
system_template = SystemMessagePromptTemplate.from_template(
    """You are a technical translator. You must maintain the exact same format and structure in your translations.
    Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

    Example input and output pairs:

    Input: "Error 404: Page not found"
    Output: "Error 404: Página no encontrada"

    Input: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operation completed'
    }}"
    Output: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operación completada'
    }}"
    """
)

# Example of a technical input
human_template = HumanMessagePromptTemplate.from_template(
    """Translate this technical information to {language}:

    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Technical Note: This error occurs when the application cannot connect to the database.
    """
)

# Create the chain using LCEL pipe syntax
chain = (
    ChatPromptTemplate.from_messages([system_template, human_template])
    | chat
    | StrOutputParser()
)

# Loop through each language
for language in languages:
    print(f"\n=== Technical Translation in {language} ===")

    # Invoke the chain with our input
    result = chain.invoke({"language": language})

    print(result)
    print("=" * 80)  # Separator for readability


=== Technical Translation in English ===
    Status: 500 Internal Server Error
    Response: {
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: This error occurs when the application cannot connect to the database.

=== Technical Translation in Esperanto ===
    Status: 500 Internal Server Error
    Response: {
        'error': 'Konekto al Database malsukcesis',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: Ĉi tiu eraro okazas kiam la aplikaĵo ne povas konekti al la database.

=== Technical Translation in Spanish ===
    Status: 500 Internal Server Error
    Response: {
        'error': 'Conexión a la base de datos fallida',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: Este error ocurre cuando la aplicación no puede conectarse a la base de datos.


Perfect, we seem to get a good response!

Now, it's arguable as to whether all of the above is better than simple f-strings like:

In [20]:
from langchain.schema import SystemMessage, HumanMessage

# Create the system message with examples
system_message = SystemMessage(content="""You are a technical translator. You must maintain the exact same format and structure in your translations.
Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

Example input and output pairs:

Input: "Error 404: Page not found"
Output: "Error 404: Página no encontrada"

Input: "Status: 200 OK
Response: {
    'data': 'success',
    'message': 'Operation completed'
}"
Output: "Status: 200 OK
Response: {
    'data': 'success',
    'message': 'Operación completada'
}"
""")

# Loop through each language
for language in languages:
    print(f"\n=== Technical Translation in {language} ===")

    # Create the human message using f-string
    human_message = HumanMessage(content=f"""Translate this technical information to {language}:

    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Technical Note: This error occurs when the application cannot connect to the database.
    """)

    # Create messages list
    messages = [system_message, human_message]

    # Get response
    res = chat.invoke(messages)

    print(res.content)
    print("=" * 80)  # Separator for readability


=== Technical Translation in English ===
    Status: 500 Internal Server Error
    Response: {
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: This error occurs when the application cannot connect to the database.

=== Technical Translation in Esperanto ===
    Status: 500 Internal Server Error
    Response: {
        'error': 'Konekto al datumbazo malsukcesis',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: Ĉi tiu eraro okazas kiam la aplikado ne povas konekti al la datumbazo.

=== Technical Translation in Spanish ===
    Status: 500 Internal Server Error
    Response: {
        'error': 'Fallo en la conexión a la base de datos',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }

    Technical Note: Este error ocurre cuando la aplicación no puede conectarse a la base de datos.


In this example, the above is far simpler. So we wouldn't necessarily recommend using prompt templates over f-strings in all scenarios.

One example where Prompt Templates might prove useful is in interpreting specific template format types. For example, suppose a project uses lots of `jinja` templates. Rather than writing our functions that handle the input values, f-strings and which renders the jinja template, LangChain Prompt Templates do all of this for us:

In [22]:
# Create few-shot examples for technical content formatting
system_template = SystemMessagePromptTemplate.from_template(
    """You are a technical translator. You must maintain the exact same format and structure in your translations.
    Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

    Example input and output pairs:

    Input: "Error 404: Page not found"
    Output: "Error 404: Página no encontrada"

    Input: "Status: 200 OK
    Response: {% raw %}{{
        'data': 'success',
        'message': 'Operation completed'
    }}{% endraw %}"
    Output: "Status: 200 OK
    Response: {% raw %}{{
        'data': 'success',
        'message': 'Operación completada'
    }}{% endraw %}"
    """,
    template_format="jinja2"
)

# Example of a technical input using Jinja2's control structures and filters
human_template = HumanMessagePromptTemplate.from_template(
    """Translate this technical information to {{ language|upper }}:

    Status: 500 Internal Server Error
    Response: {% raw %}{{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}{% endraw %}

    Technical Note: This error occurs when the application cannot connect to the database.

    {% if language == 'spanish' %}
    Note: Please use formal Spanish for technical documentation.
    {% elif language == 'french' %}
    Note: Please use formal French for technical documentation.
    {% else %}
    Note: Please maintain a formal tone in the translation.
    {% endif %}

    {% for term in technical_terms %}
    Keep the term "{{ term }}" unchanged in the translation.
    {% endfor %}
    """,
    template_format="jinja2"
)

# Create the chain using LCEL pipe syntax
chain = (
    ChatPromptTemplate.from_messages([system_template, human_template])
    | chat
    | StrOutputParser()
)

# Loop through each language
for language in languages:
    print(f"\n=== Technical Translation in {language} ===")

    # Invoke the chain with our inputs
    result = chain.invoke({
        "language": language,
        "technical_terms": ['DB_001', 'Internal Server Error', 'Database connection']
    })

    print(result)
    print("=" * 80)  # Separator for readability


=== Technical Translation in English ===
Status: 500 Internal Server Error
Response: {{
    'error': 'Database connection failed',
    'code': 'DB_001',
    'timestamp': '2024-03-20T10:30:00Z'
}}

Technical Note: This error occurs when the application cannot connect to the database.

=== Technical Translation in Esperanto ===
Status: 500 Internal Server Error
Response: {{
    'error': 'Konekto al Database connection malsukcesis',
    'code': 'DB_001',
    'timestamp': '2024-03-20T10:30:00Z'
}}

Teknika noto: Ĉi tiu eraro okazas kiam la aplikaĵo ne povas konektiĝi al la datumbazo.

Noto: Bonvolu konservi formalan tonon en la traduko.

=== Technical Translation in Spanish ===
    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection falló',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Nota técnica: Este error ocurre cuando la aplicación no puede conectarse a la database.

    
    Nota: Por favor mantenga un tono fo

Let's see what the prompts look like after LangChain interprets the Jinja2 templates. This demonstrates how LangChain automatically handles the template interpretation for us:

In [23]:
# Get the formatted prompt for Spanish
print("\n=== Formatted Prompt for Spanish ===")

# Format the prompts with our inputs
formatted_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template
]).format_prompt(
    language='spanish',
    technical_terms=['DB_001', 'Internal Server Error', 'Database connection']
)

# Print the formatted messages
for message in formatted_prompt.to_messages():
    print(f"\n{message.type.upper()} MESSAGE:")
    print("-" * 40)
    print(message.content)
    print("=" * 80)


=== Formatted Prompt for Spanish ===

SYSTEM MESSAGE:
----------------------------------------
You are a technical translator. You must maintain the exact same format and structure in your translations.
    Only translate the explanatory text, keeping all technical terms, numbers, and formatting unchanged.

    Example input and output pairs:

    Input: "Error 404: Page not found"
    Output: "Error 404: Página no encontrada"

    Input: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operation completed'
    }}"
    Output: "Status: 200 OK
    Response: {{
        'data': 'success',
        'message': 'Operación completada'
    }}"
    

HUMAN MESSAGE:
----------------------------------------
Translate this technical information to SPANISH:

    Status: 500 Internal Server Error
    Response: {{
        'error': 'Database connection failed',
        'code': 'DB_001',
        'timestamp': '2024-03-20T10:30:00Z'
    }}

    Technical Note: This error oc

Let's break down how LangChain automatically interpreted the Jinja2 templates in our prompts:

1. **Language Filter and Variable**:
   - Original: `{{ language|upper }}`
   - Interpreted as: `SPANISH`
   - The `|upper` filter automatically converted the language to uppercase

2. **Conditional Logic**:
   - Original:
     ```jinja2
     {% if language == 'spanish' %}
     Note: Please use formal Spanish for technical documentation.
     {% elif language == 'french' %}
     Note: Please use formal French for technical documentation.
     {% else %}
     Note: Please maintain a formal tone in the translation.
     {% endif %}
     ```
   - Interpreted as: `Note: Please use formal Spanish for technical documentation.`
   - The `if` statement automatically selected the Spanish-specific note

3. **Loop Structure**:
   - Original:
     ```jinja2
     {% for term in technical_terms %}
     Keep the term "{{ term }}" unchanged in the translation.
     {% endfor %}
     ```
   - Interpreted as three separate lines, one for each technical term:
     ```
     Keep the term "DB_001" unchanged in the translation.
     Keep the term "Internal Server Error" unchanged in the translation.
     Keep the term "Database connection" unchanged in the translation.
     ```
   - The `for` loop automatically iterated through our list of technical terms

4. **Raw JSON Blocks**:
   - Original: `{% raw %}{{ ... }}{% endraw %}`
   - Interpreted as: `{{ ... }}`
   - The `raw` tags were automatically removed while preserving the JSON structure